In [327]:
import yfinance as yf
import pandas as pd
import requests
from io import StringIO  # Needed to wrap the HTML string
import numpy as np



```
# This is formatted as code
```

## Module 2 Homework (2025 cohort)

In this homework, we're going to combine data from various sources to process it in Pandas and generate additional fields.



---
### Question 1: [IPO] Withdrawn IPOs by Company Type

**What is the total withdrawn IPO value (in $ millions) for the company class with the highest total withdrawal value?**

From the withdrawn [IPO list](stockanalysis.com/ipos/withdrawn), collect and process the data to find out which company type saw the most withdrawn IPO value.

**Answer**: Which class had the highest total value of withdrawals?


In [313]:
# Step 1:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

url = 'https://stockanalysis.com/ipos/withdrawn/'

response = requests.get(url, headers=headers)

# Wrap the HTML in StringIO to avoid the warning
df = pd.read_html(StringIO(response.text))[0]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Symbol          100 non-null    object
 1   Company Name    100 non-null    object
 2   Price Range     100 non-null    object
 3   Shares Offered  100 non-null    object
dtypes: object(4)
memory usage: 3.3+ KB


In [314]:
df.head()

,Symbol,Company Name,Price Range,Shares Offered
0,ODTX,"Odyssey Therapeutics, Inc.",-,-
1,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000
2,AURN,"Aurion Biotech, Inc.",-,-
3,ROTR,"PHI Group, Inc.",-,-
4,ONE,One Power Company,-,-


In [356]:
# Step 2: Create a new column called Company Class
def classify_company(name):
    clean_name = name.lower().replace(',', '')
    if "acquisition corp" in clean_name:
        return "Acq.Corp"
    elif "inc" in clean_name:
        return "Inc"
    elif "group" in clean_name:
        return "Group"
    elif "ltd" in clean_name:
        return "Limited"
    elif "limited" in clean_name:
        return "Limited"
    elif "holdings" in clean_name:
        return "Holdings"
    else:
        return "Other"

#clean_names = df['Company Name'].str.lower().str.replace(',', '')

new_col = df['Company Name'].apply(classify_company)
df['Company Class'] = new_col
df.tail()

,Symbol,Company Name,Price Range,Shares Offered,Company Class
95,FHP,"Freehold Properties, Inc.",-,-,Inc
96,CHO,Chobani Inc.,-,-,Inc
97,IFIT,iFIT Health & Fitness Inc.,$18.00 - $21.00,30769231,Inc
98,GLGX,"Gerson Lehrman Group, Inc.",-,-,Inc
99,HCG,hear.com N.V.,$17.00 - $20.00,16220000,Other


In [357]:
df['Company Class'].value_counts()

,count
Company Class,
Inc,51
Acq.Corp,21
Limited,17
Other,6
Group,4
Holdings,1


In [372]:
# Step 3: Define a new field Avg. price by parsing the Price Range field
def cal_avg_price(str):
    price_arr = str.replace('$','').replace('-','').split()
    if len(price_arr) == 1:
        return float(price_arr[0])
    elif len(price_arr) == 2:
        return (float(price_arr[0]) + float(price_arr[1])) / 2
    return None

In [374]:
df["Avg. price"] = df["Price Range"].apply(cal_avg_price).replace(np.nan, 0)
df.head()
# df['Price Range'].value_counts()[:10]

,Symbol,Company Name,Price Range,Shares Offered,Company Class,Avg. price,Withdrawn Value
0,ODTX,"Odyssey Therapeutics, Inc.",-,0.0,Inc,0.0,0.0
1,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000.0,Inc,3.5,7000000.0
2,AURN,"Aurion Biotech, Inc.",-,0.0,Inc,0.0,0.0
3,ROTR,"PHI Group, Inc.",-,0.0,Inc,0.0,0.0
4,ONE,One Power Company,-,0.0,Other,0.0,0.0


In [375]:
# Step 4: Convert Shares Offered to numeric, clean missing or invalid values.
df['Shares Offered'] = pd.to_numeric(df['Shares Offered'], errors='coerce')
# df.head(30)
df['Shares Offered'].value_counts(dropna=False).iloc[:10]

,count
Shares Offered,
0.0,28
20000000.0,9
7500000.0,4
2000000.0,3
3750000.0,3
1500000.0,2
2250000.0,2
1400000.0,2
10000000.0,2


In [414]:
# Step 5: Create a new column: Withdrawn Value = Shares Offered * Avg Price
df['Withdrawn Value'] = df['Shares Offered'] * df['Avg. price']
(df['Withdrawn Value'] > 0).value_counts()
print(f'Total Record Withdraw Value greater than 0: ', (df['Withdrawn Value'] > 0).sum())

Total Record Withdraw Value greater than 0:  71


In [420]:
# Step 6: Group by Company Class and calculate total withdrawn value.
df.groupby('Company Class')['Withdrawn Value'].sum().sort_values(ascending=False)
ans = df.groupby('Company Class')['Withdrawn Value'].sum().sort_values(ascending=False).head(1)

company_class = ans.index[0]
withdrawn_value = ans.iloc[0]

print(f"Top Company Class: {company_class}, Withdrawn Value: ${withdrawn_value:,.2f}")

Top Company Class: Acq.Corp, Withdrawn Value: $4,021,000,000.00



---
### Question 2.[IPO] Median Sharpe Ratio for 2024 IPOs (First 5 Months)

**What is the median Sharpe ratio (as of 6 June 2025) for companies that went public in the first 5 months of 2024?**

In [538]:
# Step 1: Using the same approach as in Question 1, download the IPOs in 2024 from: https://stockanalysis.com/ipos/2024/ Filter to keep only those IPOs before 1 June 2024
url = 'https://stockanalysis.com/ipos/2024/'

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

response = requests.get(url, headers=headers)

ipo_df = pd.read_html(StringIO(response.text))[0]
ipo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225 entries, 0 to 224
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      225 non-null    object
 1   Symbol        225 non-null    object
 2   Company Name  225 non-null    object
 3   IPO Price     225 non-null    object
 4   Current       225 non-null    object
 5   Return        225 non-null    object
dtypes: object(6)
memory usage: 10.7+ KB


In [539]:
ipo_df['IPO Date'] = pd.to_datetime(ipo_df['IPO Date'])
ipo_df.head()
clean_ipo_df = ipo_df[(ipo_df["IPO Date"] < "2024-06-01") & (ipo_df["IPO Price"] != '-')]
ipo_df.shape, clean_ipo_df.shape

((225, 6), (75, 6))

In [542]:
# Step 2: Use Code Snippet 7 to download daily stock data. Make sure you understand how growth_1d ... growth_365d, and volatility columns are defined.
# Define a new column growth_252d representing growth after 252 trading days (~1 year), in addition to any other growth periods you already track.
ALL_TICKERS = clean_ipo_df['Symbol'].values

print(len(ALL_TICKERS))
print(ALL_TICKERS)

import time

stocks_df = pd.DataFrame({'A' : []})

for i,ticker in enumerate(ALL_TICKERS):
#   print(i,ticker)

  # Work with stock prices
  ticker_obj = yf.Ticker(ticker)

  historyPrices = ticker_obj.history(
                     period = "max",
                     interval = "1d")

  # generate features for historical prices, and what we want to predict
  historyPrices['Ticker'] = ticker
  historyPrices['Date'] = historyPrices.index.date

  # historical returns
  for i in [1,3,7,30,90,252,365]:
    historyPrices['growth_'+str(i)+'d'] = historyPrices['Close'] / historyPrices['Close'].shift(i)
  historyPrices['growth_future_30d'] = historyPrices['Close'].shift(-30) / historyPrices['Close']

  # 30d rolling volatility : https://ycharts.com/glossary/terms/rolling_vol_30
  historyPrices['volatility'] =   historyPrices['Close'].rolling(30).std() * np.sqrt(252)

  # sleep 1 sec between downloads - not to overload the API server
  time.sleep(1)


  if stocks_df.empty:
    stocks_df = historyPrices
  else:
    stocks_df = pd.concat([stocks_df, historyPrices], ignore_index=True)

75
['BOW' 'HDL' 'RFAI' 'JDZG' 'RAY' 'BTOC' 'ZK' 'GPAT' 'PAL' 'SVCO' 'NNE'
 'CCIX' 'VIK' 'ZONE' 'LOAR' 'MRX' 'RBRK' 'NCI' 'MFI' 'YYGH' 'TRSG' 'CDTG'
 'CTRI' 'IBTA' 'MTEN' 'TWG' 'ULS' 'PACS' 'MNDR' 'CTNM' 'MAMO' 'ZBAO'
 'BOLD' 'MMA' 'UBXG' 'IBAC' 'AUNA' 'BKHA' 'LOBO' 'RDDT' 'ALAB' 'INTJ'
 'RYDE' 'LGCL' 'SMXT' 'VHAI' 'DYCQ' 'CHRO' 'UMAC' 'HLXB' 'MGX' 'TBBB'
 'TELO' 'KYTX' 'PMNT' 'AHR' 'LEGT' 'ANRO' 'GUTS' 'AS' 'FBLG' 'AVBP' 'BTSG'
 'HAO' 'CGON' 'YIBO' 'JL' 'SUGP' 'JVSA' 'KSPI' 'CCTG' 'PSBD' 'SYNX' 'SDHC'
 'ROMA']


In [543]:
# Step 3: Calculate the Sharpe ratio assuming a risk-free rate of 4.5%:
stocks_df['Sharpe'] = (stocks_df['growth_252d'] - 0.045) / stocks_df['volatility']
stocks_df[['Date', 'Sharpe']].tail()

,Date,Sharpe
23384,2025-06-16,0.541218
23385,2025-06-17,0.524219
23386,2025-06-18,0.555850
23387,2025-06-20,0.586611
23388,2025-06-23,0.588634


In [544]:
# Step 4: Filter the DataFrame to keep data only for the trading day: ‘2025-06-06’
stocks_df['Date'] = pd.to_datetime(stocks_df['Date'])
filter_df = stocks_df[stocks_df.Date == pd.to_datetime('2025-06-06')]
filter_df[['growth_252d', 'Sharpe']].describe()

print(f"Median growth_252d for 71 companies as of 2025-06-06: {filter_df['growth_252d'].median():.3f}")
print(f"Median Sharpe Ratio for 71 companies as of 2025-06-06: {filter_df['Sharpe'].median():.3f}")

Median growth_252d for 71 companies as of 2025-06-06: 0.758
Median Sharpe Ratio for 71 companies as of 2025-06-06: 0.084


In [545]:
# Step 5: [Additional] Do you observe the same top 10 companies when sorting by growth_252d versus sorting by Sharpe?
top10_growth252d = filter_df.sort_values('growth_252d', ascending=False)[['Ticker', 'growth_252d']].head(10)
top10_sharpe = filter_df.sort_values('Sharpe', ascending=False)[['Ticker','Sharpe']].head(10)

print("Top 10 Companies by growth_252d:")
print(top10_growth252d)

print("\nTop 10 Companies by Sharpe:")
print(top10_sharpe)

print("\nTop 10 Companies by growth_252d and Sharpe are different.")

Top 10 Companies by growth_252d:
      Ticker  growth_252d
20545     JL     8.097413
23378   ROMA     6.156406
14333   UMAC     4.966533
2966     NNE     4.655224
4662    RBRK     3.184065
16730    AHR     2.483097
18081     AS     2.478203
4372     MRX     2.300384
11553   RDDT     2.225505
7010    MTEN     2.210432

Top 10 Companies by Sharpe:
      Ticker    Sharpe
10927   BKHA  2.835668
21221   JVSA  2.041531
17039   LEGT  1.940267
10337   IBAC  1.637119
14675   HLXB  1.123493
8206    MNDR  0.974234
13658   DYCQ  0.969321
12183   INTJ  0.744512
20545     JL  0.566222
5834    TRSG  0.518080

Top 10 Companies by growth_252d and Sharpe are different.



---
### Question 3. [IPO] ‘Fixed Months Holding Strategy’


**What is the optimal number of months (1 to 12) to hold a newly IPO'd stock in order to maximize average growth?** (Assume you buy at the close of the first trading day and sell after a fixed number of trading days.)

Goal:
Investigate whether holding an IPO stock for a fixed number of months after its first trading day produces better returns, using future growth columns.

In [548]:
# Step 1: Start from 75 tickers from IPOs in the first 5 months of 2024 from Q2. Add 12 new columns:future_growth_1m to 12m. Similar to Snippet 7, but extend to longer timeframes.
future_df = pd.DataFrame({'A' : []})

for i,ticker in enumerate(ALL_TICKERS):
#  print(i,ticker)

  # Work with stock prices
  ticker_obj = yf.Ticker(ticker)

  historyPrices = ticker_obj.history(
                     period = "max",
                     interval = "1d")

  # generate features for historical prices, and what we want to predict
  historyPrices['Ticker'] = ticker
  historyPrices['Date'] = historyPrices.index.date

  # historical returns
  for i in range(1,13):
      historyPrices[f'future_growth_{i}m'] = historyPrices['Close'].shift(-i*21) / historyPrices['Close']
  # sleep 1 sec between downloads - not to overload the API server
  time.sleep(1)


  if future_df.empty:
    future_df = historyPrices
  else:
    future_df = pd.concat([future_df, historyPrices], ignore_index=True)

In [549]:
# Step 2: Determine the first trading day (min_date) for each ticker. Earliest date in the data for each stock.
min_date = future_df.groupby('Ticker')['Date'].min()
min_date_df = pd.DataFrame(min_date)
min_date_df.head()


,Date
Ticker,
AHR,2024-02-07
ALAB,2024-03-20
ANRO,2024-02-02
AS,2024-02-01
AUNA,2024-03-22


In [550]:
# Step 3: Join the data: Perform an inner join between the min_date DataFrame and the future growth data on both ticker and date.
# need to reset idx to bring Ticker to columns
min_date_df_reset = min_date_df.reset_index()
merged = pd.merge(future_df, min_date_df_reset, on=['Ticker', 'Date'], how='inner')
merged

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker,Date,future_growth_1m,...,future_growth_3m,future_growth_4m,future_growth_5m,future_growth_6m,future_growth_7m,future_growth_8m,future_growth_9m,future_growth_10m,future_growth_11m,future_growth_12m
0,23.000000,24.270000,22.139999,23.799999,3335800,0.0,0.0,BOW,2024-05-23,1.024790,...,1.301681,1.167647,1.239076,1.461765,1.443698,1.357983,1.389076,1.722689,1.698319,1.533613
1,27.000000,30.000000,19.799999,22.290001,670000,0.0,0.0,HDL,2024-05-17,0.775236,...,0.673845,0.643786,0.720502,0.719605,1.255271,1.106326,1.030507,1.192014,0.907133,1.002602
2,10.000000,10.080000,10.000000,10.040000,1175800,0.0,0.0,RFAI,2024-07-05,1.002191,...,1.007968,1.014940,1.014940,1.017928,1.022908,1.030877,1.035857,1.040837,1.046813,NaN
3,4.800000,5.400000,3.595000,4.070000,2366200,0.0,0.0,JDZG,2024-05-15,0.242998,...,0.122850,0.160442,0.272727,0.199017,0.139066,0.182555,0.170025,0.146437,0.121867,0.142506
4,5.050000,6.040000,4.020000,4.230000,2379300,0.0,0.0,RAY,2024-05-15,0.839243,...,0.368794,0.463357,0.397163,0.406619,0.395508,0.290780,0.297872,0.255319,0.602837,0.432624
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,5.750000,9.833000,5.030000,7.700000,5181000,0.0,0.0,CCTG,2024-01-18,0.467532,...,0.311688,0.316883,0.279221,0.290909,0.206494,0.225974,0.232468,0.196104,0.198701,0.228571
71,14.039482,14.220914,13.832130,13.987644,1044800,0.0,0.0,PSBD,2024-01-18,1.016059,...,1.062494,1.060584,1.061857,1.081292,1.066809,1.074739,1.096368,1.063983,1.060609,1.095498
72,3.990000,4.000000,3.260000,3.610000,711400,0.0,0.0,SYNX,2024-01-12,0.994460,...,1.002770,0.664820,0.753463,0.761773,0.844875,0.844875,0.822715,0.767313,0.825485,1.177285
73,23.820000,24.670000,23.500000,24.150000,426100,0.0,0.0,SDHC,2024-01-16,1.117598,...,1.104348,1.209938,1.036025,1.130849,1.426915,1.484886,1.548654,1.285714,1.340373,1.024017


In [551]:
# Step 4: Compute descriptive statistics for the resulting DataFrame: Use .describe() or similar to analyze each of the 12 columns:
# Select only the future growth columns
growth_columns = [f'future_growth_{i}m' for i in range(1, 13)]
growth_df = future_df[growth_columns]

# Get statistics
stats = growth_df.describe()

# Display the result
print(stats)

       future_growth_1m  future_growth_2m  future_growth_3m  future_growth_4m  \
count      21814.000000      20239.000000      18664.000000      17089.000000   
mean           1.033897          1.059764          1.070942          1.090109   
std            0.784107          1.053486          1.161033          1.267875   
min            0.020233          0.011224          0.008863          0.013852   
25%            0.853250          0.762733          0.686032          0.617834   
50%            0.993498          0.970000          0.945690          0.925532   
75%            1.080000          1.096306          1.107418          1.118694   
max           33.944955         23.888887         21.764706         16.454082   

       future_growth_5m  future_growth_6m  future_growth_7m  future_growth_8m  \
count      15514.000000      13939.000000      12364.000000      10789.000000   
mean           1.105828          1.118486          1.113058          1.102862   
std            1.349977    

In [552]:
# Step 5: Determine the best holding period: Find the number of months (1 to 12) where the average (mean) future growth is maximal.
# This optimal month shows an uplift of >1% compared to all others. Still, the average return remains less than 1 (i.e., expected return is less than doubling your investment).
max_mean_month = stats.loc['mean'].idxmax()
max_mean_value = stats.loc['mean'].max()
print(f"The best holding period to maximize growth is", max_mean_month[-2], "months. With a mean growth of", round(max_mean_value, 3))

The best holding period to maximize growth is 6 months. With a mean growth of 1.118



---
### Question 4.  [Strategy] Simple RSI-Based Trading Strategy


**What is the total profit (in $thousands) you would have earned by investing $1000 every time a stock was oversold (RSI < 25)?** Goal:
Apply a simple rule-based trading strategy using the Relative Strength Index (RSI) technical indicator to identify oversold signals and calculate profits.

In [553]:
# Run the full notebook from Lecture 2 (33 stocks)
# Step 1 & 2: Download precomputed data using this snippet:
import gdown
import pandas as pd

file_id = "1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-"
gdown.download(f"https://drive.google.com/uc?id={file_id}", "data.parquet", quiet=False)
df = pd.read_parquet("data.parquet", engine="pyarrow")

Downloading...
From (original): https://drive.google.com/uc?id=1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-
From (redirected): https://drive.google.com/uc?id=1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-&confirm=t&uuid=dd42e640-0f4c-4ca3-8d76-2d789be80afa
To: /content/data.parquet
100%|██████████| 130M/130M [00:01<00:00, 72.6MB/s]


In [566]:
# Step 3: RSI Strategy Setup: RSI is already available in the dataset as a field. The threshold for oversold is defined as RSI < 25.
# Step 4: Filter the dataset by RSI and date:
rsi_threshold = 25
selected_df = df[
    (df['rsi'] < rsi_threshold) &
    (df['Date'] >= '2000-01-01') &
    (df['Date'] <= '2025-06-01')
]
print(f"Total number of trades: {len(selected_df)}")

Total number of trades: 1568


In [571]:
# Step 5: What is the net income in $K (i.e., in thousands of dollars) that could be earned using this RSI-based oversold strategy from 2000–2025?
net_income = 1000 * (selected_df['growth_future_30d'] - 1).sum()
net_income
print("The net income could be earned using this RSI-based oversold strategy from 2000-2025 is", round(net_income / 1000, 2), "thousands of dollars.")

The net income could be earned using this RSI-based oversold strategy from 2000-2025 is 24.3 thousands of dollars.


### Question 5.[Exploratory, Optional] Predicting a Positive-Return IPO

**Most of the strategies for investing in IPOs deliver negative average and median returns (and even 75% quantiles).** Question:
How would you change the strategy if you want to increase the profitability?

In [ ]:
# Ans: Try to target IPOs with strong fundamentals (profitability, revenue growth, competitive edge).